# Transform API Response into CSV

In [21]:
# import libraries 
import os
import json
import sys
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd


sys.path.insert(0, str(Path().resolve().parent / "src"))

from paths import  PARENT_DIR, TRANSFORMED_DATA_DIR, RAW_DATA_DIR


In [55]:
def load_and_transform_json_file_to_dataframe(raw_json_file) -> pd.DataFrame:
    # Define file path for the current day's data
    
    if raw_json_file.suffix == ".json":
        
        flatten_data = []

        print(f"Loading file {raw_json_file}")
        with open(raw_json_file, "r") as f:
            data = json.load(f)
        
            if 'list' in data:
                for items in data["list"]:
                    aqi = items["main"]["aqi"]
                    co = items["components"]["co"]
                    no = items["components"]["no"]
                    no2 = items["components"]["no2"]
                    o3 = items["components"]["o3"]
                    so2 = items["components"]["so2"]
                    pm2_5 = items["components"]["pm2_5"]
                    pm10 = items["components"]["pm10"]
                    nh3 = items["components"]["nh3"]
                    timestamp = datetime.utcfromtimestamp(items["dt"]).strftime("%Y-%m-%d %H:%M:%S")
                    
                    flatten_data.append({
                        "aqi":aqi, "co":co, "no":no, "no2":no2, "o3":o3, "so2":so2, 
                        "pm2_5":pm2_5, "pm10":pm10, "nh3":nh3, "timestamp":timestamp
                    })

                
                df = pd.DataFrame(flatten_data)
                
                row_ids = [row_id for row_id in range(len(df))]
                df.insert(0, "row_id", row_ids )
                
                if not Path(TRANSFORMED_DATA_DIR).exists():
                    os.mkdir(TRANSFORMED_DATA_DIR)
                
                transform_file_name = raw_json_file.name.split(".")[0]
                file_path = TRANSFORMED_DATA_DIR / f"{transform_file_name}.csv"
                
                # Save dataframe response to a file
                df.to_csv(file_path, index=False)
                
                print(f"Data successfully transformed and saved to {file_path}")
            else:
                print(f"Unexpected structure in {raw_json_file}")
    else:
        print(f"File {raw_json_file} not found.")

In [56]:
if __name__ == "__main__":
    for raw_file in RAW_DATA_DIR.iterdir():
        load_and_transform_json_file_to_dataframe(raw_file)

Loading file /Users/macbook/Desktop/horlarDEV/Air_MLOPs/data/bronze/weather_20200101_to_20250201.json
Data successfully transformed and saved to /Users/macbook/Desktop/horlarDEV/Air_MLOPs/data/gold/weather_20200101_to_20250201.csv


In [57]:
df = pd.read_csv(f"{TRANSFORMED_DATA_DIR}/weather_20200101_to_20250201.csv")
df.head()

,row_id,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,timestamp
0,0,5,1682.28,0.13,18.85,12.88,8.82,64.62,90.85,17.48,2020-11-25 01:00:00
1,1,5,2109.53,0.36,21.94,9.30,10.37,93.95,127.43,21.03,2020-11-25 02:00:00
2,2,5,2750.40,1.41,26.39,4.16,12.52,136.28,181.39,25.59,2020-11-25 03:00:00
3,3,5,3337.86,4.81,28.45,0.78,14.07,175.09,233.20,28.63,2020-11-25 04:00:00
4,4,5,3738.40,10.95,28.45,0.10,15.26,200.27,262.51,30.91,2020-11-25 05:00:00


In [58]:
df.shape

(36311, 11)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36311 entries, 0 to 36310
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   row_id     36311 non-null  int64  
 1   aqi        36311 non-null  int64  
 2   co         36311 non-null  float64
 3   no         36311 non-null  float64
 4   no2        36311 non-null  float64
 5   o3         36311 non-null  float64
 6   so2        36311 non-null  float64
 7   pm2_5      36311 non-null  float64
 8   pm10       36311 non-null  float64
 9   nh3        36311 non-null  float64
 10  timestamp  36311 non-null  object 
dtypes: float64(8), int64(2), object(1)
memory usage: 3.0+ MB


In [60]:
# Convert timestamp column to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Extract date and time into separate columns
df['date'] = df['timestamp'].dt.date
df['time'] = df['timestamp'].dt.time

In [61]:
def aqi_mapping_category(feature: int) -> str:
    """
    Maps an Air Quality Index (AQI) numeric value to its corresponding category.

    Parameters:
    feature (int): AQI value ranging from 1 to 5.

    Returns:
    str: Corresponding AQI category ("Good", "Fair", "Moderate", "Poor", "Very Poor") 
         or "Undefined" for invalid values.
    """
    categories = {
        1: "Good",
        2: "Fair",
        3: "Moderate",
        4: "Poor",
        5: "Very Poor"
    }
    return categories.get(feature, "Undefined")

df["aqi_bucket"] = df["aqi"].apply(aqi_mapping_category)
df.head()

,row_id,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,timestamp,date,time,aqi_bucket
0,0,5,1682.28,0.13,18.85,12.88,8.82,64.62,90.85,17.48,2020-11-25 01:00:00,2020-11-25,01:00:00,Very Poor
1,1,5,2109.53,0.36,21.94,9.30,10.37,93.95,127.43,21.03,2020-11-25 02:00:00,2020-11-25,02:00:00,Very Poor
2,2,5,2750.40,1.41,26.39,4.16,12.52,136.28,181.39,25.59,2020-11-25 03:00:00,2020-11-25,03:00:00,Very Poor
3,3,5,3337.86,4.81,28.45,0.78,14.07,175.09,233.20,28.63,2020-11-25 04:00:00,2020-11-25,04:00:00,Very Poor
4,4,5,3738.40,10.95,28.45,0.10,15.26,200.27,262.51,30.91,2020-11-25 05:00:00,2020-11-25,05:00:00,Very Poor


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36311 entries, 0 to 36310
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   row_id      36311 non-null  int64         
 1   aqi         36311 non-null  int64         
 2   co          36311 non-null  float64       
 3   no          36311 non-null  float64       
 4   no2         36311 non-null  float64       
 5   o3          36311 non-null  float64       
 6   so2         36311 non-null  float64       
 7   pm2_5       36311 non-null  float64       
 8   pm10        36311 non-null  float64       
 9   nh3         36311 non-null  float64       
 10  timestamp   36311 non-null  datetime64[ns]
 11  date        36311 non-null  object        
 12  time        36311 non-null  object        
 13  aqi_bucket  36311 non-null  object        
dtypes: datetime64[ns](1), float64(8), int64(2), object(3)
memory usage: 3.9+ MB


In [63]:
df.head()

,row_id,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,timestamp,date,time,aqi_bucket
0,0,5,1682.28,0.13,18.85,12.88,8.82,64.62,90.85,17.48,2020-11-25 01:00:00,2020-11-25,01:00:00,Very Poor
1,1,5,2109.53,0.36,21.94,9.30,10.37,93.95,127.43,21.03,2020-11-25 02:00:00,2020-11-25,02:00:00,Very Poor
2,2,5,2750.40,1.41,26.39,4.16,12.52,136.28,181.39,25.59,2020-11-25 03:00:00,2020-11-25,03:00:00,Very Poor
3,3,5,3337.86,4.81,28.45,0.78,14.07,175.09,233.20,28.63,2020-11-25 04:00:00,2020-11-25,04:00:00,Very Poor
4,4,5,3738.40,10.95,28.45,0.10,15.26,200.27,262.51,30.91,2020-11-25 05:00:00,2020-11-25,05:00:00,Very Poor


In [64]:
# Aqi unique values
df['aqi'].value_counts()

aqi
5    23016
4     6788
3     4468
2     2033
1        6
Name: count, dtype: int64

In [65]:
df.to_csv(f"{TRANSFORMED_DATA_DIR}/weather_20200101_to_20250201.csv", index=False)